<a href="https://colab.research.google.com/github/PFSV/bspl-ku.github.io/blob/main/bAbi_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import tarfile
from nltk import FreqDist
from functools import reduce
import os
import re
import matplotlib.pyplot as plt

In [2]:
path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/'
                'babi_tasks_1-20_v1-2.tar.gz')

11755520/11745123 [==============================] - 0s 0us/step


In [4]:
with tarfile.open(path) as tar: 
 tar.extractall()
 tar.close()

DATA_DIR = 'tasks_1-20_v1-2/en-10k'
TRAIN_FILE = os.path.join(DATA_DIR, "qa1_single-supporting-fact_train.txt")
TEST_FILE = os.path.join(DATA_DIR, "qa1_single-supporting-fact_test.txt")


In [52]:

'''
i = 0
lines = open(TRAIN_FILE , "rb")
for line in lines:
    line = line.decode("utf-8").strip()
    # lno, text = line.split(" ", 1) # ID와 TEXT 분리
    i = i + 1
    print(line)
    if i == 20:
      break
'''

# Open file 
with open(TRAIN_FILE) as f:
    textlist = f.readlines()
num = 0
for i in textlist:
  print(i)
  num += 1
#print("total number of text is ",num) # Total number of sentence is 30000


Streaming output truncated to the last 5000 lines.
6 Where is Mary? 	office	4

7 Daniel travelled to the office.

8 Daniel went back to the bedroom.

9 Where is Sandra? 	bathroom	2

10 John journeyed to the kitchen.

11 Sandra went back to the kitchen.

12 Where is John? 	kitchen	10

13 Daniel journeyed to the bathroom.

14 Daniel went back to the garden.

15 Where is Sandra? 	kitchen	11

1 Sandra went back to the bedroom.

2 Sandra went to the garden.

3 Where is Sandra? 	garden	2

4 Daniel travelled to the bathroom.

5 Mary moved to the garden.

6 Where is Mary? 	garden	5

7 Mary went back to the bathroom.

8 John moved to the kitchen.

9 Where is John? 	kitchen	8

10 Mary went to the kitchen.

11 John travelled to the office.

12 Where is Mary? 	kitchen	10

13 Mary went to the bedroom.

14 Mary travelled to the bathroom.

15 Where is Mary? 	bathroom	14

1 Daniel went back to the bedroom.

2 Daniel travelled to the office.

3 Where is Daniel? 	office	2

4 Mary went back to the bathro

Reading w/ "rb"
=> reading with binary mode

Example
with open(filename,"rb") as f:
    print(f.read())# 출력
b'this file is for test.\r\n\xc0\xcc\xc6\xc4\xc0\xcf\xc0\xba \xc5\xd7\xbd\xba\xc6\xae\xbf\xeb \xc0\xd4\xb4\xcf\xb4\xd9.'


In [47]:
def read_data(dir):
    stories, questions, answers = [], [], [] #list for stories, question, answers
    story_temp = [] # list for temporary story
    lines = open(dir, "rb") 



    for line in lines:
        line = line.decode("utf-8") #need to transfer binary mode file into human-readable(especially for space and tab keys)  
        line = line.strip() # '\n' 제거
        idx, text = line.split(" ", 1) # 맨 앞에 있는 id number 분리
        # 여기까지는 모든 줄에 적용되는 전처리

        if int(idx) == 1:
            story_temp = []

        if "\t" in text: # 현재 읽는 줄이 질문 (tab) 답변 (tab)인 경우
            question, answer, _ = text.split("\t") # 질문과 답변을 각각 저장
            stories.append([x for x in story_temp if x]) # 지금까지의 누적 스토리를 스토리에 저장
            questions.append(question)
            answers.append(answer)

        else: # 현재 읽는 줄이 스토리인 경우
            story_temp.append(text) # 임시 저장

    lines.close()
    return stories, questions, answers


In [ ]:
def read_data_hyunseop(dir):
    stories, questions, answers = [], [], [] #list for stories, question, answers
    story_temp = [] # list for temporary story
    with open(TRAIN_FILE) as f:
      textlist = f.readlines() #['1 Mary moved to the bathroom.\n', '2 John went to the hallway.\n', '3 Where is Mary? \tbathroom\t1\n',
      for i in textlist:
        i = i.split("\n")
    

# Open file 
with open(TRAIN_FILE) as f:
    textlist = f.readlines()
num = 0
for i in textlist:
  print(i)
  num += 1
#print("total number of text is ",num) # Total number of sentence is 30000

In [109]:
def read_data_hyunseop(dir):
    stories, questions, answers = [], [], [] #list for stories, question, answers
    story_now = []
    #Preprocessing
    with open(TRAIN_FILE) as f:
      textlist = f.readlines()  #['1 Mary moved to the bathroom.\n', '2 John went to the hallway.\n', '3 Where is Mary? \tbathroom\t1\n',
      for i in textlist:
        i = i.split("\n") #need to split by each sentence
        i = i[ : -1] #delete remaining vestiges of \n
        for p in i:
          idnum, text = p.split(" ",1) #split number and text
          
          if idnum == "1":
            story_now = [] # make a new list for every story begins
          if "\t" in text: # question, answer, answer_clue sentence are located where "\t" are in
            question,answer,_ = text.split("\t") # split question,answer
            questions.append(question) # collect all questions
            answers.append(answer)     # collect all answer
            stories.append([x for x in story_now if x])#append accumulated stories
          else: # append stories if it is not the question,answering sentence
            story_now.append(text)

    f.close()
    return stories,questions,answers


In [110]:
train_data = read_data_hyunseop(TRAIN_FILE)
test_data = read_data_hyunseop(TEST_FILE)

In [111]:
train_stories,train_questions,train_answers = read_data_hyunseop(TRAIN_FILE)
test_stories,test_questions,test_answers = read_data_hyunseop(TEST_FILE)

In [112]:
print('훈련용 스토리의 개수 :', len(train_stories))
print('훈련용 질문의 개수 :',len(train_questions))
print('훈련용 답변의 개수 :',len(train_answers))
print('테스트용 스토리의 개수 :',len(test_stories))
print('테스트용 질문의 개수 :',len(test_questions))
print('테스트용 답변의 개수 :',len(test_answers))

훈련용 스토리의 개수 : 10000
훈련용 질문의 개수 : 10000
훈련용 답변의 개수 : 10000
테스트용 스토리의 개수 : 10000
테스트용 질문의 개수 : 10000
테스트용 답변의 개수 : 10000


In [118]:
train_stories[2820] # Test whether the story, question, answer is correct

['Mary went to the hallway.', 'John went back to the bathroom.']

In [119]:
train_questions[2820]

'Where is John? '

In [120]:
train_answers[2820]

'bathroom'

In [121]:
train_questions[3576]

'Where is John? '

In [122]:
train_answers[3576]


'bedroom'